# **Problema A - Caso Base**

# Matriz de Costos

In [14]:
import pandas as pd

df_clients = pd.read_csv('./content/caso3/clients.csv')
df_depots = pd.read_csv('./content/base/depots.csv') # deposito del caso base
df_vehicles = pd.read_csv('./content/caso3/vehicles.csv')

In [15]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [16]:
import openrouteservice
from openrouteservice import convert
import json

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)

matrix = client.distance_matrix(
    locations=lugares,
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

ApiError: 400 ({'error': {'code': 6004, 'message': 'Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed.'}, 'info': {'engine': {'build_date': '2025-05-06T08:31:01Z', 'graph_version': '1', 'graph_date': '2025-05-25T13:10:00Z', 'version': '9.2.0'}, 'timestamp': 1748348905363}})

In [7]:
import numpy as np
dist_matrix   =matrix['distances']

precio_km = 3032.1
cost = {}
distance = {}

# Opción B: sin tocar los DataFrames originales, solo con listas
depot_labels  = ['CD{}'.format(d) for d in df_depots['DepotID']]
client_labels = ['C{}'.format(c)  for c in df_clients['ClientID']]

# Genera la lista de labels (primero depósitos, luego clientes)
labels = depot_labels + client_labels

# 2. Convierte la matriz de distancias en un DataFrame
distance_df = pd.DataFrame(dist_matrix, index=labels, columns=labels)

# 3. Calcula la matriz de costos multiplicando por precio_km
cost_df = distance_df * precio_km

np.fill_diagonal(cost_df.values, 9999999)
cost_df.head()

,CD1,C1,C2,C3,C4,C5,C6,C7,C8,C9
CD1,9999999.000,81290.601,92024.235,49969.008,81078.354,35536.212,49544.514,72770.400,58701.456,67130.694
C1,104819.697,9999999.000,24560.010,78349.464,9247.905,73467.783,40054.041,80350.650,68252.571,43601.598
C2,93237.075,24287.121,9999999.000,66766.842,29290.086,61885.161,41115.276,69586.695,56669.949,32837.643
C3,56093.850,56184.813,67797.756,9999999.000,55972.566,24741.936,41721.696,48513.600,19405.440,42024.906
C4,104577.129,7519.608,25651.566,78106.896,9999999.000,73255.536,39811.473,80108.082,68010.003,43359.030


In [11]:
filename = 'cost_matrix_caso_2.csv'

# Guarda con encabezado (labels) pero sin índice pandas
cost_df.to_csv(filename, index=False)

In [13]:
import csv
import numpy as np
def read_cost_matrix_csv(filename):
    """
    Reads a cost matrix from a CSV file.

    The first row of the CSV is assumed to contain the node IDs.

    Parameters:
        filename (str): The name of the CSV file.
    
    Returns:
        cost_matrix (np.array): A 2D numpy array representing the cost matrix.
    """
    with open(filename, 'r', newline='') as file:
        reader = csv.reader(file)

        # Skip the header row with node IDs
        next(reader)

        data = list(reader)

    cost_matrix = np.array(data, dtype=float)

    return cost_matrix

cost_m = pd.read_csv('cost_matrix_caso_2.csv')
cost_m

,CD1,C1,C2,C3,C4,C5,C6,C7,C8,C9
0,9999999.000,81290.601,92024.235,49969.008,81078.354,35536.212,49544.514,72770.400,58701.456,67130.694
1,104819.697,9999999.000,24560.010,78349.464,9247.905,73467.783,40054.041,80350.650,68252.571,43601.598
2,93237.075,24287.121,9999999.000,66766.842,29290.086,61885.161,41115.276,69586.695,56669.949,32837.643
3,56093.850,56184.813,67797.756,9999999.000,55972.566,24741.936,41721.696,48513.600,19405.440,42024.906
4,104577.129,7519.608,25651.566,78106.896,9999999.000,73255.536,39811.473,80108.082,68010.003,43359.030
5,38841.201,57488.616,68252.571,25105.788,57306.690,9999999.000,43055.820,48998.736,34899.471,43328.709
6,44814.438,33444.063,36627.768,44147.376,33231.816,45633.105,9999999.000,65826.891,49878.045,30957.741
7,78015.933,77015.340,62400.618,51939.873,76833.414,46664.019,65129.508,9999999.000,31928.013,34262.730
8,64856.619,69101.559,64098.594,30563.568,68889.312,33535.026,52000.515,30684.852,9999999.000,37961.892
9,76863.735,41479.128,34687.224,50393.502,41266.881,45542.142,31533.840,35414.928,40296.609,9999999.000
